In [ ]:
import torch
import torch.nn as nn
from torch import Tensor
from transformers import GPTNeoXForCausalLM, AutoTokenizer, AutoModelForCausalLM
from jaxtyping import Float, Int
from typing import List, Optional, Tuple, Dict
import sys
from functools import partial
from tqdm import tqdm
import json
import seaborn as sns
import pandas as pd
import multiprocessing
import pickle
import transformer_lens.utils as utils
import plotly.express as px
import einops
import numpy as np
import plotly.io as pio
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float

import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

from request_patching import request_patch_one_pair, create_patch_request_dict, baseline_completion, baseline_completion_plus
from models import get_model_from_name

import warnings
#warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device =', device)
torch.set_grad_enabled(False)

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")

else:
    print("CUDA is not available. Listing CPUs instead.")
    print(multiprocessing.cpu_count())

torch.set_grad_enabled(False)

ModuleNotFoundError: No module named 'circuitsvis'

In [10]:
!pip install circuitsvis

INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  You can safely remove it manually.
  Attempting uninstall: torch
    Found existing installation: torch 2.2.0
    Uninstalling torch-2.2.0:
      Succe

In [3]:
def imshow(tensor, **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    ).show()


def line(tensor, **kwargs):
    px.line(
        y=utils.to_numpy(tensor),
        **kwargs,
    ).show()


def scatter(x, y, xaxis="", yaxis="", caxis="", **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x": xaxis, "y": yaxis, "color": caxis},
        **kwargs,
    ).show()

In [ ]:
model = HookedTransformer.from_pretrained(
    "pythia-6.9b",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

device: torch.device = utils.get_device()

NameError: name 'HookedTransformer' is not defined

Verify that the model is able to do the task

In [8]:
def us_election_prompts(year: str):
    """
    2000 - Alice: Gore, Bob: Bush
    2008 - Alice: Obama, Bob: McCain
    """
    
    context_1 = f"""Alice: Have you been following the election coverage? I can't believe it's already November {year}. This election feels more significant than any I can remember. 
Bob: Yes, I've been keeping a close eye on it. This year, as a committed Republican, I find the election particularly pivotal. The Republican agenda, focusing on economic stability and national security, really resonates with me.
Alice: I'm siding with the Democrats myself. Their emphasis on healthcare and education matches what I believe our country desperately needs right now.
Bob: I understand where you're coming from, Alice. However, from my standpoint, the Republican principles of limited government and individual freedoms are what we need to navigate these challenging times.
Alice: I know exactly who I will vote for. I support Mr."""

    context_2 = f"""Alice: Have you been following the election coverage? I can't believe it's already November {year}. This election feels more significant than any I can remember. 
Bob: Yes, I've been keeping a close eye on it. This year, as a committed Republican, I find the election particularly pivotal. The Republican agenda, focusing on economic stability and national security, really resonates with me.
Alice: I'm siding with the Democrats myself. Their emphasis on healthcare and education matches what I believe our country desperately needs right now.
Bob: I understand where you're coming from, Alice. However, from my standpoint, the Republican principles of limited government and individual freedoms are what we need to navigate these challenging times.
Alice: I see your point, Bob. Regardless, {year} feels like a watershed moment.
Bob: I know exactly who I will vote for. I support Mr."""

    return context_1, context_2

In [ ]:
example_prompt = us_election_prompts('2000')[0]
example_answer = " Mary"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

In [ ]:
prompt_format = [
    us_election_prompts('2000')[0],
    us_election_prompts('2000')[1],
    us_election_prompts('1992')[0],
    us_election_prompts('1992')[1],
]
names = [
    (" Mary", " John"),
    (" Tom", " James"),
    (" Dan", " Sid"),
    (" Martin", " Amy"),
]
# List of prompts
prompts = []
# List of answers, in the format (correct, incorrect)
answers = []
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = []
for i in range(len(prompt_format)):
    for j in range(2):
        answers.append((names[i][j], names[i][1 - j]))
        answer_tokens.append(
            (
                model.to_single_token(answers[-1][0]),
                model.to_single_token(answers[-1][1]),
            )
        )
        # Insert the *incorrect* answer to the prompt, making the correct answer the indirect object.
        prompts.append(prompt_format[i].format(answers[-1][1]))
answer_tokens = torch.tensor(answer_tokens).to(device)
print(prompts)
print(answers)